# Preppin' Data 2023W04

Source: https://preppindata.blogspot.com/2023/01/2023-week-4-new-customers.html

### Load file

In [132]:
import pandas as pd
import datetime as dt

Because some months had different headers, I appended without header names then set the headers based on the January worksheet

In [76]:
#df0 = pd.read_excel('New Customers.xlsx',sheet_name='January')

In [114]:
#df = pd.concat(pd.read_excel('New Customers.xlsx',sheet_name=None, header=None), ignore_index=True)

In [110]:
#df.columns[0:3] = df0.columns

In [137]:
# read file, ignore column headers and sheet names
df = pd.read_excel('New Customers.xlsx',sheet_name=None, header=None)

In [138]:
# add a new column which is the sheet name
df = pd.concat([sheet.assign(Month=i) for i, sheet in df.items()])

In [139]:
# rename columns
df = df.rename(columns={0:'ID',1:'Joining Day',2:'Demographic',3:'Value'})

In [140]:
# drop the first row as the column names were not right
df = df.iloc[1:]

In [154]:
df

,ID,Joining Day,Demographic,Value,Month,Year
1,490910,3,Ethnicity,White,1,2023
2,490910,3,Date of Birth,5/23/1981,1,2023
3,490910,3,Account Type,Basic,1,2023
4,369221,18,Ethnicity,Black,1,2023
5,369221,18,Date of Birth,3/4/2019,1,2023
...,...,...,...,...,...,...
269,174699,2,Date of Birth,3/13/1989,12,2023
270,174699,2,Account Type,Gold,12,2023
271,514598,28,Ethnicity,Other,12,2023
272,514598,28,Date of Birth,10/10/1971,12,2023


### Make Joining Day field

In [142]:
df['Month'] = pd.to_datetime(df['Month'], format='%B').dt.month.astype(str)

In [159]:
df['Year'] = 2023

In [173]:
df['Day'] = pd.to_numeric(df['Joining Day'])

ValueError: Unable to parse string "Joining Day" at position 249

In [179]:
# index position 249 caused error so dropped row
df = df.drop(df.index[249])

In [180]:
df['Joining Date'] = pd.to_datetime(df[['Year','Month','Day']])

In [181]:
df

,ID,Joining Day,Demographic,Value,Month,Year,Day,Joining Date
1,490910,3,Ethnicity,White,1,2023,3,2023-01-03
2,490910,3,Date of Birth,5/23/1981,1,2023,3,2023-01-03
3,490910,3,Account Type,Basic,1,2023,3,2023-01-03
4,369221,18,Ethnicity,Black,1,2023,18,2023-01-18
5,369221,18,Date of Birth,3/4/2019,1,2023,18,2023-01-18
...,...,...,...,...,...,...,...,...
269,174699,2,Date of Birth,3/13/1989,12,2023,2,2023-12-02
270,174699,2,Account Type,Gold,12,2023,2,2023-12-02
271,514598,28,Ethnicity,Other,12,2023,28,2023-12-28
272,514598,28,Date of Birth,10/10/1971,12,2023,28,2023-12-28


### Reshape data

In [193]:
df2 = df.pivot_table(values='Value', index='ID', columns='Demographic', aggfunc='first').reset_index()

In [206]:
df3 = pd.merge(df, df2, how='left', on=['ID', 'ID'])

In [207]:
df3

,ID,Joining Day,Demographic,Value,Month,Year,Day,Joining Date,Account Type,Date of Birth,Ethnicity
0,490910,3,Ethnicity,White,1,2023,3,2023-01-03,Basic,5/23/1981,White
1,490910,3,Date of Birth,5/23/1981,1,2023,3,2023-01-03,Basic,5/23/1981,White
2,490910,3,Account Type,Basic,1,2023,3,2023-01-03,Basic,5/23/1981,White
3,369221,18,Ethnicity,Black,1,2023,18,2023-01-18,Platinum,3/4/2019,Black
4,369221,18,Date of Birth,3/4/2019,1,2023,18,2023-01-18,Platinum,3/4/2019,Black
...,...,...,...,...,...,...,...,...,...,...,...
2965,174699,2,Date of Birth,3/13/1989,12,2023,2,2023-12-02,Gold,3/13/1989,Asian
2966,174699,2,Account Type,Gold,12,2023,2,2023-12-02,Gold,3/13/1989,Asian
2967,514598,28,Ethnicity,Other,12,2023,28,2023-12-28,Platinum,10/10/1971,Other
2968,514598,28,Date of Birth,10/10/1971,12,2023,28,2023-12-28,Platinum,10/10/1971,Other


### Remove duplicates

In [209]:
df3 = df3.drop(['Demographic','Joining Day','Month','Year','Day', 'Value'], axis=1)

In [210]:
df3 = df3.drop_duplicates()

In [211]:
df3

,ID,Joining Date,Account Type,Date of Birth,Ethnicity
0,490910,2023-01-03,Basic,5/23/1981,White
3,369221,2023-01-18,Platinum,3/4/2019,Black
6,568991,2023-01-16,Gold,4/27/1979,White
9,191607,2023-01-09,Gold,3/15/1963,White
12,612794,2023-01-29,Basic,11/12/1981,Asian
...,...,...,...,...,...
2955,970548,2023-12-12,Platinum,5/9/1996,Asian
2958,569923,2023-12-04,Gold,12/30/1975,Other
2961,354147,2023-12-20,Gold,4/14/2019,Black
2964,174699,2023-12-02,Gold,3/13/1989,Asian


In [216]:
df3.loc[df3['ID'] == '929308']

,ID,Joining Date,Account Type,Date of Birth,Ethnicity
222,929308,2023-01-05,Basic,10/19/2004,Other


### Export

In [218]:
df3.to_csv('2023W04_output.csv', index=False)